# <div align="center"> k12学习框架错误码 </div>

## 一. 错误码描述

|错误码|简述|中文描述|详述|
|:-----:|:------|:------|:---------|
| 100000 | Success | 操作成功 | 无
| 100100 | 100100 | 预留,表示api调用传递json参数是正确的 | 无
| 100101 | Json key not found | Json串中无key关键字 | 不固定
| 100102 | Json value is invalid | Json串中value值非法 | 不固定
| 100103 | | |
| 100104 | | |
| 100105 | | |
| 100200 | 100200 | 预留, 表示启动服务无错误发生 | 无
| 100201 | Service is not found | 服务名错误,或者服务没启动 | 服务名
| 100202 | Service start fail | 服务启动失败 | 描述具体启动失败原因 
| 100203 | | |
| 100204 | | |
| 100205 | | |
| 100300 | 100300 | 预留, k12cv container 内部进程错误 |
| 100400 | 100400 | 预留, k12nlp container 内部进程错误 |
| 100401 | Container is not found! | Container没找到, 停止一个任务时可能会触发 | 
| 100402 | Container is already running! | Container已经运行, 当启动一个任务时可能触发 |
| 100500 | 100500 | |
| 100600 | 100600 | |

## 二. 案例

In [9]:
import requests
import json
import consul
import time

hostname = 'gamma'
host = '10.255.20.227'
port = 8129

user = 'test'
uuid = '123456'

consul_addr = host
consul_port = 8500

# API异步接口将过程中产出的数据同步到consul数据中心, 调用该接口可从中心获取数据 (数据中心仅供开发使用)
def query_async_data(key, wait = 1):
    time.sleep(wait)
    try:
        client = consul.Consul(consul_addr, consul_port)
        _, data = client.kv.get(key)
        return str(data['Value'], encoding="utf-8")
    except:
        return 'Query nothing, try again later!'

In [3]:
config = '''{
    "dataset_reader":{
        "type": "sst_tokens",
        "use_subtrees": true,
        "granularity": "2-class"
    },
    "validation_dataset_reader":{
        "type": "sst_tokens",
        "use_subtrees": false,
        "granularity": "2-class"
    },
    "train_data_path": "/data/datasets/nlp/sst/train.txt",
    "validation_data_path": "/data/datasets/nlp/sst/dev.txt",
    "test_data_path": "/data/datasets/nlp/sst/test.txt",  
    "model": {
        "type": "basic_classifier",        
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 300,          
                    "trainable": false          
                }
            }
        },
        "seq2vec_encoder": {
            "type": "lstm",                     
            "input_size": 300,
            "hidden_size": 512,
            "num_layers": 2,
            "batch_first": true
        }
    },    
    "iterator": {
        "type": "bucket",
        "sorting_keys": [["tokens", "num_tokens"]],
        "batch_size" : 32
    },
    "trainer": {
        "num_epochs": 5,
        "patience": 1,
        "grad_norm": 5.0,
        "validation_metric": "+accuracy",
        "cuda_device": 0,
        "optimizer": {
            "type": "adam",
            "lr": 0.001
        }
    }
}'''

### 1. 缺少参数

In [5]:
data = '''{
    "xxxop":"train.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
start_res = requests.post(url=api, json=json.loads(data))
start_res.text

'{"code": 100101, "brief": "Json key not found.", "detail": "\'op\'"}'

### 2. 参数值错误

In [6]:
data = '''{
    "op":"xxx.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
start_res = requests.post(url=api, json=json.loads(data))
start_res.text

'{"code": 100102, "brief": "Json value is invalid."}'

### 3. 服务不存在

In [7]:
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "xxxk12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
start_res = requests.post(url=api, json=json.loads(data))
start_res.text

'{"code": 100201, "brief": "Service is not found.", "detail": {"service_name": "xxxk12nlp"}}'

### 4. 服务启动失败

In [23]:
data = '''{
    "op":"train.start",
    "user": "%s",
    "service_name": "k12nlp",
    "service_uuid": "%s",
    "service_params": %s
}''' % (user, uuid, config)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
start_res = requests.post(url=api, json=json.loads(data))
start_res.text

'{"code": 100000, "detail": {"op": "train.start", "exec": "success", "cache_dir": "/data/users/test/123456"}}'

In [24]:
print(query_async_data('framework/%s/%s/train.start/error'%(user, uuid)))

{
    "version": "0.1.0",
    "type": "error",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "123456",
    "timestamp": 1574345220956,
    "datetime": "2019-11-21 22:07:00",
    "error": {
        "code": 100499,
        "brief": "404 Client Error: Not Found (\"pull access denied for hzcsai_com/xxxx, repository does not exist or may require 'docker login': denied: requested access to the resource is denied\")",
        "detail": "Traceback (most recent call last):\n  File \"/home/lidong/.local/lib/python3.6/site-packages/docker/api/client.py\", line 261, in _raise_for_status\n    response.raise_for_status()\n  File \"/home/lidong/.local/lib/python3.6/site-packages/requests/models.py\", line 940, in raise_for_status\n    raise HTTPError(http_error_msg, response=self)\nrequests.exceptions.HTTPError: 404 Client Error: Not Found for url: http+docker://localhost/v1.35/containers/create?name=train-test-123456\n\nDuring handling of the above exceptio